In [2]:
import boa

In [3]:
math_contract = boa.load("../../contracts/experimental/CurveCryptoMath2MAX.vy")
math_contract

<../../contracts/experimental/CurveCryptoMath2MAX.vy at 0x0880cf17Bd263d3d3a5c09D2D86cCecA3CcbD97c, compiled with vyper-0.3.10+9136169>

In [5]:
output = math_contract.newton_D()

In [7]:
K0_iter = output[0]
K0_iter = output[1]
_g1k0_iter = output[2]
mul1_iter = output[3]
mul2_iter = output[4]
neg_fprime_iter = output[5]
D_plus_iter = output[6]
D_minus_iter = output[7]
D_iter = output[8]

255

In [8]:
D_iter = output[-1]
len(D_iter)

255

In [9]:
D_iter

[2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 2000000000000000000000000000000000,
 